In [48]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import IPython.display as ipd
from PIL import Image
from tensorflow.keras.preprocessing import image
from tensorflow import keras
import time

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [31]:
model_eye = keras.models.load_model("eye_detection.h5")
model_yawn = keras.models.load_model("yawn_detection1.h5")

In [50]:
video_path = "10fps_laura.mp4"  
output_video_path="output.mp4"
ipd.Video(video_path, width=700)

In [51]:
#capturing frames
cap = cv2.VideoCapture(video_path)
num_frames=cap.get(cv2.CAP_PROP_FRAME_COUNT)
num_frames

1891.0

In [52]:
ret, img = cap.read()
print(f'Returned {ret} and img of shape {img.shape}')

Returned True and img of shape (720, 1280, 3)


In [53]:
#code for detecting eyes region from face
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

In [56]:
yawn_count = 0
head_rotation_count = 0
closed_eyes_count = 0
concentration_levels = {0: "Low Concentration", 1: "Medium Concentration", 2: "High Concentration"}

In [57]:
while cap.isOpened():
    ret, img = cap.read()

    #check if the video capture was successful
    if not ret:
        print("Error: Could not read frame")
        break

    #if the frame is not empty
    if img is None:
        print("Error: Empty frame")
        break

    #resize the frame to (256, 256) and convert to grayscale
    resized_frame = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), (256, 256))

    #recognition of face and eye
    for (x, y, w, h) in faces:
        #draw a rectangle around the face
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)

        #get the region for eye detection within the detected face
        roi_gray = gray[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex, ey, ew, eh) in eyes:
            #draw a rectangle around the eyes
            cv2.rectangle(img, (x + ex, y + ey), (x + ex + ew, y + ey + eh), (0, 255, 0), 2)

        #get the region for mouth detection within the detected face
        roi_gray_mouth = gray[y:y+h, x:x+w]
        mouths = mouth_cascade.detectMultiScale(roi_gray_mouth)
        for (mx, my, mw, mh) in mouths:
            # Draw a rectangle around the mouth (within the face region)
            cv2.rectangle(img, (x + mx, y + my), (x + mx + mw, y + my + mh), (0, 0, 255), 2)

    #eye detection
    if len(eyes) > 0:q
        # Resize the eyes region to (256, 256) and add an additional dimension
        resized_eyes = np.expand_dims(cv2.resize(roi_gray, (256, 256)), axis=-1)
        eye_pred = np.argmax(model_eye.predict(np.expand_dims(resized_eyes, axis=0)))
    else:
        eye_pred = -1
        cv2.putText(img, "Head rotation: Detected", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    #yawn detection
    yawn_pred = np.argmax(model_yawn.predict(resized_frame.reshape(1, 256, 256, 1)))

    #running the models
    if eye_pred == 0:
        cv2.putText(img, "Eyes: Closed", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    else:
        cv2.putText(img, "Eyes: Open", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    if yawn_pred == 0:
        cv2.putText(img, "Yawn: Detected", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        yawn_count += 1
    else:
        cv2.putText(img, "Yawn: Not Detected", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    
     #check concentration level
    if yawn_count > 5 and head_rotation_count > 5 and closed_eyes_count > 10:
        concentration_level = 0  # Low Concentration
    elif yawn_count > 3 and head_rotation_count > 3 and closed_eyes_count > 5:
        concentration_level = 1  # Medium Concentration
    elif 1 <= yawn_count <= 2 and 1 <= head_rotation_count <= 2 and closed_eyes_count > 3:
        concentration_level = 2  # High Concentration
    else:
        concentration_level = -1  # Not classified

    #display the concentration level
    cv2.putText(img, f"Concentration: {concentration_levels.get(concentration_level)}",
                (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    
    cv2.imshow('Frame', img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 62ms/step


1/1 [==============================] - 0s 46ms/step


1/1 [==============================] - 0s 77ms/step


1/1 [==============================] - 0s 43ms/step


1/1 [==============================] - 0s 71ms/step


...

1/1 [==============================] - 0s 105ms/step


1/1 [==============================] - 0s 51ms/step


1/1 [==============================] - 0s 87ms/step


1/1 [==============================] - 0s 47ms/step


1/1 [==============================] - 0s 53ms/step
Error: Could not read frame
